In [5]:
import pandas as pd
import pyterrier as pt
if not pt.started():
  pt.init()
indexref = pt.IndexRef.of("/path/to/data.properties")

In [161]:
topics.iloc[55]

qid                                         57
query    are gender or racial quotas effective
Name: 55, dtype: object

In [9]:
from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles = []
title_ids = []
nums = []
for topic in data:
    titles.append(topic.find('title').text.lower()[:-1])
    num = topic.find('number').text
    nums.append(num)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles.append(topic.find('title').text.lower()[:-1])
    num = topic.find('number').text
    nums.append(num)
topics = pd.DataFrame()
topics['qid'] = nums
topics['query'] = titles

qrels_path=("/workspace/src/evaluate_targers/qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)


In [12]:
path_targer = '/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/'
indexref_orig = pt.IndexRef.of(path_targer + "pd_index_original-100/data.properties")

index = pt.IndexFactory.of(indexref_orig)
retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"})
res_orig=retr.transform(topics)

res_orig

,qid,docid,docno,rank,score,query
0,1,129720,S51530f3f-Ae32a4a1b,0,17.318396,should teachers get tenure
1,1,88639,Sb0680508-Aa5189771,1,17.225787,should teachers get tenure
2,1,309458,Sc065954f-Ae72bc9c6,2,17.173398,should teachers get tenure
3,1,309456,Sc065954f-A24a16870,3,17.149872,should teachers get tenure
4,1,318126,Sff0947ec-A46d54897,4,17.113795,should teachers get tenure
...,...,...,...,...,...,...
98995,100,256729,S2ea519c-Aed35ffc8,995,3.581751,do we need cash
98996,100,29220,Sa68e9864-Af79ef325,996,3.580354,do we need cash
98997,100,231260,Sfb30ce24-Aa94950e7,997,3.580354,do we need cash
98998,100,294707,S78819fc5-Ae076e471,998,3.576581,do we need cash


In [14]:
eval_orig = pt.Utils.evaluate(res_orig,qrels,metrics=["ndcg_cut_5", "bpref", "ndcg_cut_25"])
eval_orig

{'ndcg_cut_5': 0.5527540851759042,
 'bpref': 0.6261477227722415,
 'ndcg_cut_25': 0.45749865160257747}

In [21]:
!ls /mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/pools_topics_conclusionspd_index

data.direct.bf		   data.lexicon.fsomaphash  data.meta.zdata
data.document.fsarrayfile  data.lexicon.fsomapid    data.properties
data.inverted.bf	   data.meta-0.fsomapfile
data.lexicon.fsomapfile    data.meta.idx


In [25]:
path_deepct = '/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/pools_topics_conclusions'
indexref_deepct = pt.IndexRef.of(path_deepct + "pd_index/data.properties")

index = pt.IndexFactory.of(indexref_deepct)
retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"})
res_deepct=retr.transform(topics)

res_deepct

,qid,docid,docno,rank,score,query
0,1,88183,Sb0680508-Aa5189771,0,21.050909,should teachers get tenure
1,1,307911,Sc065954f-Ae72bc9c6,1,21.050689,should teachers get tenure
2,1,307909,Sc065954f-A24a16870,2,21.050064,should teachers get tenure
3,1,307908,Sc065954f-A6deb09b6,3,21.047903,should teachers get tenure
4,1,129070,S51530f3f-Ae32a4a1b,4,21.044753,should teachers get tenure
...,...,...,...,...,...,...
98995,100,359294,Se6ed5e12-Aadcc223b,995,3.607130,do we need cash
98996,100,222899,S65984782-A4ff2c4e7,996,3.607039,do we need cash
98997,100,260639,Scbc1612f-A202f2817,997,3.606938,do we need cash
98998,100,293440,S3e420ef6-Acebcb2a7,998,3.606915,do we need cash


In [26]:
eval_deepct = pt.Utils.evaluate(res_deepct,qrels,metrics=["ndcg_cut_5", "bpref", "ndcg_cut_25"])
eval_deepct

{'ndcg_cut_5': 0.48994807438609406,
 'bpref': 0.7227362009170157,
 'ndcg_cut_25': 0.45310523282418635}

In [32]:
res_deepct.groupby(['qid'])

In [91]:
docnos = qrels.docno
docnos_in_orig = [res_orig['docno'].head(25).isin(docnos)]
docnos_in_orig

[0     False
 1     False
 2     False
 3     False
 4     False
 5     False
 6     False
 7     False
 8     False
 9     False
 10    False
 11    False
 12    False
 13    False
 14    False
 15    False
 16    False
 17     True
 18    False
 19    False
 20    False
 21    False
 22    False
 23    False
 24    False
 Name: docno, dtype: bool]

In [147]:
docnos_in_deepct = res_deepct['docno'].head(25).isin(docnos).to_list()
# docnos_in_deepct

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [126]:
res_orig.iloc[17]

qid                               1
docid                        309460
docno           Sc065954f-Ac3a1cfc1
rank                             17
score                       14.8194
query    should teachers get tenure
Name: 17, dtype: object

In [39]:
res_deepct.iloc[8]

qid                               1
docid                        307913
docno           Sc065954f-Ac3a1cfc1
rank                              8
score                       20.9985
query    should teachers get tenure
Name: 8, dtype: object

In [149]:
for i in range(100):
#     print(i)
    if True in (res_deepct[res_deepct['qid'] == str(i)]['docno']).head(25).isin(docnos).to_list():
        print(i)
#         print((res_orig[res_orig['qid'] == str(i)]['docno'].head(25)).isin(docnos))
        

1
4
6
7
13
15
24
28
34
37
41
42
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [215]:
for docno in (res_deepct[res_deepct['qid'] == str(58)]['docno']).head(25).to_list():
    print(qrels[qrels['docno'] == docno]['label'])

2231    0
Name: label, dtype: int64
2205    2
Name: label, dtype: int64
2219    2
Name: label, dtype: int64
2221    0
Name: label, dtype: int64
2233    2
Name: label, dtype: int64
Series([], Name: label, dtype: int64)
2203    0
Name: label, dtype: int64
Series([], Name: label, dtype: int64)
2206    0
Name: label, dtype: int64
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
2207    2
Name: label, dtype: int64
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)


In [216]:
for docno in (res_orig[res_orig['qid'] == str(58)]['docno']).head(25).to_list():
    print(qrels[qrels['docno'] == docno]['label'])

2207    2
Name: label, dtype: int64
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
2204    1
Name: label, dtype: int64
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
2203    0
Name: label, dtype: int64
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
2213    0
Name: label, dtype: int64
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
2193    0
Name: label, dtype: int64
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)
Series([], Name: label, dtype: int64)


In [164]:
topics.iloc[56]['query']

'should holders of public offices resign on bad approval ratings'

In [203]:
(res_deepct[res_deepct['qid'] == str(58)]['docno']).head(25).isin(docnos).to_list()

[True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False]

In [165]:
(res_orig[res_orig['qid'] == str(58)]['docno']).head(25).isin(docnos).to_list()

[True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False]

In [217]:
qrels[qrels['qid'] == str(58)]

,qid,docno,label
2193,58,Sc98ea55b-A7de6603e,0
2194,58,Sb8c1b53b-A983f3ea4,0
2195,58,S88057b35-Aae9df0b3,0
2196,58,Scee72b1d-Ab8645f14,0
2197,58,S37e9fe07-Aa8e11a88,0
2198,58,Sf78850e9-A5ee430bc,0
2199,58,S8472bcef-A1fc8e6bf,0
2200,58,Sabe4d584-Ab09d1a44,0
2201,58,S2c813efa-A97c94a4,0
2202,58,S5e356d3a-Ab0a24a9f,0


In [226]:
(res_deepct[res_deepct['qid'] == str(58)]['docno']).to_list().index('Sce1fe489-Aa0395e1d')

25

In [227]:
(res_orig[res_orig['qid'] == str(58)]['docno']).to_list().index('Sce1fe489-Aa0395e1d')

3

In [218]:
(res_deepct[res_deepct['qid'] == str(58)]['docno']).head(7)

56000     S1396dd39-A727f34c
56001    S1396dd39-A3e98eb46
56002    S1396dd39-A2f82a344
56003    S148784b7-A64092cb2
56004    S1396dd39-A3ea31951
56005    S6fc99c30-Ae718e9a8
56006    S9dd3f3ac-A99a8f66d
Name: docno, dtype: object

In [199]:
top25_deepct = (res_deepct[res_deepct['qid'] == str(58)]['docno']).head(25).to_list()
top25_orig = (res_orig[res_orig['qid'] == str(58)]['docno']).head(25).to_list()

missed_by_orig = []

for docno in docnos:
    if docno in top25_deepct and docno not in top25_orig:
        missed_by_orig.append(docno)
        print(docno)

S1396dd39-A3e98eb46
S148784b7-Aa0d9d97
S1396dd39-A2f82a344
S148784b7-A64092cb2
S1396dd39-A727f34c
S1396dd39-A3ea31951


In [175]:
top25_deepct = (res_deepct[res_deepct['qid'] == str(58)]['docno']).head(25).to_list()
top25_orig = (res_orig[res_orig['qid'] == str(58)]['docno']).head(25).to_list()

missed = []

for docno in docnos:
    if docno in top25_orig and docno not in top25_deepct:
        missed.append(docno)
        print(docno)

Sc98ea55b-A7de6603e
Sce1fe489-Aa0395e1d
Sc98ea55b-Ad7eb21fe


In [171]:
print([docno for docno in docnos if docno in top25_deepct])
print([docno for docno in docnos if docno in top25_orig])

['S9dd3f3ac-A99a8f66d', 'S1396dd39-A3e98eb46', 'S148784b7-Aa0d9d97', 'Sb0fc3466-A5a151ade', 'S1396dd39-A2f82a344', 'S148784b7-A64092cb2', 'S1396dd39-A727f34c', 'S1396dd39-A3ea31951']
['Sc98ea55b-A7de6603e', 'S9dd3f3ac-A99a8f66d', 'Sce1fe489-Aa0395e1d', 'Sb0fc3466-A5a151ade', 'Sc98ea55b-Ad7eb21fe']


In [176]:
founded = []

for docno in docnos:
    if docno not in top25_orig and docno in top25_deepct:
        founded.append(docno)
        print(docno)

S1396dd39-A3e98eb46
S148784b7-Aa0d9d97
S1396dd39-A2f82a344
S148784b7-A64092cb2
S1396dd39-A727f34c
S1396dd39-A3ea31951


In [186]:
from tqdm import tqdm
import ir_datasets
dataset = ir_datasets.load('argsme/2020-04-01')
for doc in tqdm(dataset.docs_iter(), total=387740):

    if doc.doc_id in missed:
        print(f'missed : {doc.source_url, doc.premises_texts}')
    elif doc.doc_id in founded:
          print(f'founded : {doc.source_url, doc.premises_texts}')

  2%|▏         | 7120/387740 [00:00<00:10, 35070.82it/s]

founded : ('https://www.debate.org/debates/If-a-president-has-an-approval-rating-of-33-then-they-should-honorably-resign./1/', 'You are "shocked" by what I say only because you have some misconceptions about this country, and about democracy in general. While we are taught in school that democracy necessarily equals freedom, and that democratic nations are free nations that is not always the case. My job is not to provide you with an alternative as to which we could judge the performance of politicians, but simply prove yours wrong. While immediate inspection may show that democracy is a great way to determine which is good, this is not the case. In fact, I will try to show that it is an especially bad way of deciding what is good and what is bad, and that we should not use democracy, in any respect, to decide if our leaders should resign or remain the leaders. While our country does use a somewhat democratic process to decide who our president will be, this is not a very good process 

  4%|▎         | 14456/387740 [00:00<00:10, 35753.47it/s]

founded : ('https://www.debate.org/debates/Anthony-Weiner-should-not-have-resigned-his-seat-in-Congress/1/', '1) "...Anthony Weiner is a Congressmen[.] [H]e is supposed to represent his voters with integrity, respect, and leadership." A politician strictly represents voters politically! His lack of integrity in his marriage has not been proven to have anything do with his integrity in regards to his voters. Yes, his act may cause voters to think more lowly of him, but that just means that he would have a tough time running for re-election, if he were to seek it. It does not mean, though, that he should resign from his current office.The fact of the matter is that no charges were filed against him, and he was not impeached. Hence, there was no legal obligation for him to resign. 2) Weiner\'s approval rating dropped, yes. But who\'s to say that he couldn\'t have recovered and made his approval rating skyrocket? I don\'t see how his having a low approval rating immediately after his scand

 44%|████▍     | 171763/387740 [00:04<00:05, 37406.78it/s]

missed : ('https://www.debate.org/debates/A-Moderate-President-Would-Be-Much-More-Likely-to-have-Positive-Approval-Ratings/1/', "Sorry but that's not very much of an argument. A shaky opinion at best. What proof do you have? Also, president FDR, on the complete different end from Reagan, had one of the highest approval ratings of any president while in office. He is the only president to have ever been elected more than twice. In-fact he was elected 4 times. History doesn't seem to serve you justice.")
missed : ('https://www.debate.org/debates/A-Moderate-President-Would-Be-Much-More-Likely-to-have-Positive-Approval-Ratings/1/', "First I'm going to assume that we're talking about US presidents. Second, I'd like to point out that president Ronald Reagan had one of the largest approval ratings of any US president while in office and was extremely far from being a moderate. Third, I will point out that my opponent has the burden of proof. I need prove nothing but rather negate supposed pro

 68%|██████▊   | 262493/387740 [00:07<00:03, 37309.15it/s]

missed : ('https://www.debate.org/debates/Bad-politicians-should-face-deportation-from-the-US-when-their-terms-end/1/', "Ladies and Gentlemen, the US government has a problem. Ok they have a sh*tload of problems but one of them stands out more than just about anything else. Its not the threat of nuclear war or inflation or unemployment or Decepticons........ No I am talking about bad politicians...... There is a saying in Israel that everyone on both sides want peace, but that all it takes is for one zealot to mess up everything. The same concept can be applied to the American government, where the few good politicians we have cannot bring America the change it needs because of the actions of one a**hole always undo's any progress made. What becomes of these bad politicians when they're term ends? bad publicity, low poll ratings, and that's about it. This nation needs to take a tougher stance against bad politicians, and I would recommend that they all be violated with a loaded shotgun

100%|██████████| 387740/387740 [00:10<00:00, 35683.22it/s]


In [198]:
#docnos that are retrieved both by original dataset and deepct but not considered relevant 
overlap = [docno for docno in top25_orig if ((docno in top25_deepct) and (docno not in docnos))]
print(f'number of documents both retrieved by deepct and original dataset, but not considered relevant : {len(overlap)} from 25')

overlap

number of documents both retrieved by deepct and original dataset, but not considered relevant : 7 from 25


['Sb0fc3466-A5a151ade',
 'S5c22eb90-A7cc8f4ff',
 'S6fc99c30-Ae718e9a8',
 'S80b17c10-A8144f295',
 'S9dd3f3ac-A99a8f66d',
 'S81403cf6-A1a77b26e',
 'Se4117131-Aaac9d692']

In [228]:
first_unrelevant_deepct = 'S6fc99c30-Ae718e9a8'
for doc in tqdm(dataset.docs_iter(), total=387740):
    if doc.doc_id == first_unrelevant_deepct:
        print(f'first_unrelevant_deepct : {doc.source_url, doc.premises_texts}')

 98%|█████████▊| 379275/387740 [00:10<00:00, 31270.03it/s]

first_unrelevant_deepct : ('https://debatewise.org/debates/2853-drug-testing-mandatory-for-public-officials/', 'Drugs are illegal. Holders of public office and those elected to power should set an example by demonstrating that they do not break the law. We have a right to hold our leaders to a higher standard. And if they are breaking the law, the public is entitled to know about it.')


100%|██████████| 387740/387740 [00:10<00:00, 36544.36it/s]


In [229]:
arg_relevance_one = 'Sce1fe489-Aa0395e1d'
for doc in tqdm(dataset.docs_iter(), total=387740):
    if doc.doc_id == arg_relevance_one:
        print(f'arg_relevance_one : {doc.source_url, doc.premises_texts}')

 68%|██████▊   | 262443/387740 [00:06<00:03, 38313.42it/s]

arg_relevance_one : ('https://www.debate.org/debates/Bad-politicians-should-face-deportation-from-the-US-when-their-terms-end/1/', "Ladies and Gentlemen, the US government has a problem. Ok they have a sh*tload of problems but one of them stands out more than just about anything else. Its not the threat of nuclear war or inflation or unemployment or Decepticons........ No I am talking about bad politicians...... There is a saying in Israel that everyone on both sides want peace, but that all it takes is for one zealot to mess up everything. The same concept can be applied to the American government, where the few good politicians we have cannot bring America the change it needs because of the actions of one a**hole always undo's any progress made. What becomes of these bad politicians when they're term ends? bad publicity, low poll ratings, and that's about it. This nation needs to take a tougher stance against bad politicians, and I would recommend that they all be violated with a loa

100%|██████████| 387740/387740 [00:10<00:00, 36692.03it/s]
